In [2]:
from sparse_format import SparseRepresentFormat
from sparse_align import SparseAlign
from sparse_addr_convert import SparseAddrConvert
from sparse_skip import SparseSkip
from sparse_tile import SparseTile


from utils.sparse_matrix import SparseMatrix

import numpy as np


class SparseUnit: 
    def __init__(self, ):
        self.a = 0
    

 
    def top(self, sparse_matrix_a, matrix_b, one_side_sparsity, 
            align_strategy, skip_strategy, tile_strategy,
            systolic_size=(16, 16)):

        A_access_msg = []
        B_access_msg = []

        # =============================================================================        
        # align: 稀疏对齐，获得访存矩阵
        align = SparseAlign(sparse_matrix_a, matrix_b, A_sparsity=True, \
                                 B_sparsity=~one_side_sparsity, mode=align_strategy)
        
        a_align_access, b_align_access = align.a_seq, align.b_seq
        
        # 返回COO格式的序列
        # with open("./a_align_access.txt", "w") as f:
        #     for r, c, v in zip(a_align_access.row, a_align_access.col, a_align_access.data):
        #         f.write(f"{r} {c} {v}\n")
        
        # with open("./b_align_access.txt", "w") as f:
        #     for r, c, v in zip(b_align_access.row, b_align_access.col, b_align_access.data):
        #         f.write(f"{r} {c} {v}\n")
        # =============================================================================        
        # skip: 通过skip让计算与访存更紧凑

        # =============================================================================        
        # addr_convert: 相对地址转为矩阵的绝对地址(仍为虚拟地址)

        # =============================================================================        
        # tile: 不同平铺策略得到最终访存pattern
        

        
        return A_access_msg, B_access_msg
        
    

    
if __name__ == '__main__':
    A_sparse = SparseMatrix(2048, 2048, 1/16).matrix
    # for i in A_sparse[0]:
        # print(A_sparse[0][i], end='')
    csr_A_sparse = SparseRepresentFormat(A_sparse, 'csr').sparse_matrix
    # print(csr_A_sparse.sparse_matrix)
    B_dense = np.random.randint(0, 10, size=(128, 2048))
    
    # A_access_msg, B_access_msg= SparseUnit().top(sparse_matrix_a=csr_A_sparse, 
    #     matrix_b=B_dense, one_side_sparsity=True, \
    #     align_strategy="indirection", skip_strategy="two_dim_skip", tile_strategy="horizontal_tile", \
    #     systolic_size=(16, 16))
    
    
        

A_sparse = SparseMatrix(2048, 2048, 1/16).matrix
csr_A_sparse = SparseRepresentFormat(A_sparse, 'csr').sparse_matrix
# print(csr_A_sparse)
B_dense = np.random.randint(0, 10, size=(128, 2048))
    


align = SparseAlign(csr_A_sparse, B_dense, A_sparsity=True, \
                                 B_sparsity=False, mode="indirection")
        
a_align_access, b_align_access = align.a_seq, align.b_seq # COO格式

# a_align_access.row
# with open("./a_align_access.txt", "w") as f:
#     for r, c, v in zip(a_align_access.row, a_align_access.col, a_align_access.data):
#         f.write(f"{r} {c} {v}\n")
        
# with open("./b_align_access.txt", "w") as f:
#     for r, c, v in zip(b_align_access.row, b_align_access.col, b_align_access.data):
#         f.write(f"{r} {c} {v}\n")
b_toarray = b_align_access.toarray()
# np.set_printoptions(threshold=np.inf)
# with open("./b_toarray.txt", "w") as f:
#     for line in b_toarray:
#        line_str = np.array2string(line, separator=', ', max_line_width=np.inf)
#        f.write(line_str + "\n")  # 将每一行写入文件，手动换行
# np.set_printoptions(threshold=1000)
# 对大矩阵做一次二维skip
# a_skip1_access = SparseSkip().two_dim_skip(a_align_access)
# b_skip1_access = SparseSkip().two_dim_skip(b_align_access)
from scipy.sparse import coo_matrix

systolic_size=(16, 16)
systolic_row, systolic_col = systolic_size

# 2048 x2048
a_toarray = a_align_access.toarray()

a_new_arry = np.zeros((a_toarray.shape), dtype=object)
# col_dim_skip
for i in range(0, a_toarray.shape[0], systolic_row):
    skip_block = SparseSkip().col_dim_skip(coo_matrix(a_toarray[i:i+systolic_row])) # 对齐
    # print(skip_block)
    # 如果当前列数小于 2048，则在右侧用 -1 填充到 2048 列
    if skip_block.shape[1] < 2048:
        skip_block = np.pad(skip_block, ((0, 0), (0, 2048 - skip_block.shape[1])), constant_values=-1)
    a_new_arry[i:i+systolic_row] = skip_block
    # print(a_new_arry)

np.set_printoptions(threshold=np.inf)
np.set_printoptions(formatter={'all': lambda x: f"{x}"})
with open("./a_toarray.txt", "w") as f:
    for line in a_new_arry:
        # print(line)
        line_str = np.array2string(line, separator=', ', max_line_width=np.inf)
        # print(line_str)
        f.write(line_str + "\n")  # 将每一行写入文件，手动换行
np.set_printoptions(threshold=1000)
# 计算PE个数
# a_skip1_access.shape
# b_skip1_access.shape
block_row_id = 0
PE_id = 0
PE_tile = [] 

for i in range(0, a_toarray.shape[0], systolic_row):
    a_tiles = SparseTile().sliding_window(a_new_arry[i:i+systolic_row], window_size=systolic_size)
    a_tiles = a_tiles[::systolic_col, ::systolic_col] # 一行的PEs
    # print(a_tiles.shape[1])
    for j in range(a_tiles.shape[1]):
            # 检查当前 tile 是否全是 -1
        if np.all(a_tiles[0][j] == -1):
            break
        else:
            PE_tile.append((block_row_id, PE_id, a_tiles[0][j]))
            PE_id += 1  
    block_row_id += 1

np.set_printoptions(threshold=np.inf)
with open("./PE_tiles.txt", "w") as f:
    for block_row_id, pe_id, tile in PE_tile:
        tile_str = np.array2string(tile, separator=', ', max_line_width=np.inf)
        # 将 PE_id 和 tile 一起写入文件
        f.write(f"block_row_id: {block_row_id} PE_id: {pe_id}" + "\n" + tile_str + "\n\n")  # 加一个换行来分隔不同的 tiles
np.set_printoptions(threshold=1000)


PE_tile_coo = []
for block_row_id, pe_id, tile in PE_tile:
    # 直接将 tile 转换为 COO 矩阵
    tile_coo = coo_matrix(tile)
    
    # 将 (block_row_id, pe_id, tile_coo) 存入 PE_tile_coo 列表
    PE_tile_coo.append((block_row_id, pe_id, tile_coo))

# 输出 COO 格式的 tiles
# for block_row_id, pe_id, tile_coo in PE_tile_coo:
#     print(f"Block Row ID: {block_row_id}, PE ID: {pe_id}")
#     print(f"COO Matrix:\n{tile_coo}")

In [30]:
np.set_printoptions(threshold=np.inf)
with open("./B_mem_access.txt", "w") as f:
    for block_row_id, pe_id, tile_coo in PE_tile_coo:
        f.write(f"[block_row_id: {block_row_id}, PE_id: {pe_id}], ")
        for row, col, value in zip(tile_coo.row, tile_coo.col, tile_coo.data):
            # if pe_id== 0:
            #     print(row, col, value)
            #     print(systolic_row)
            if value == -1: 
                continue
            (row_i, col_i) = value
            f.write(str(col_i) + ", ")     

        # b_vaddr = (row * systolic_row + col) + (pe_id * systolic_row * systolic_col)  # 相对位置+绝对位置
            # a的多少列对应b的多少行,一行128
        f.write(" \n")     
np.set_printoptions(threshold=1000)



# 获取a的相对位置并访存
np.set_printoptions(threshold=np.inf)


global_counter = 1  # 全局编码从 1 开始
encoded_values = []  # 用于保存编码后的值和对应位置

# 遍历每个 block_row
for block_row_id in range(max([block_row_id for block_row_id, _, _ in PE_tile]) + 1):
    block_row_pes = [tile for row_id, pe_id, tile in PE_tile if row_id == block_row_id]
    
    if len(block_row_pes) == 0:
        continue
    
    # 每个 block_row 内一共 16 行，假设所有 PE 的行数相同
    num_rows = block_row_pes[0].shape[0]
    num_cols = block_row_pes[0].shape[1]

    # 依次遍历 block_row 的每一行
    for i in range(num_rows):
        # 遍历 block_row 中每个 PE 的第 i 行
        for pe_id, tile in enumerate(block_row_pes):
            # 由于是 dtype=object, tile[i] 是一个对象, 比如一个 ndarray
            tile_row = tile[i]  # 获取第 i 行的对象
            for j in range(num_cols):
                if tile_row[j] != 0 and tile_row[j] != -1:
                    # 将 block_row_id, pe_id, 行号 i, 列号 j 以及全局编码保存
                    encoded_values.append((block_row_id, pe_id, i, j, global_counter))
                    global_counter += 1  # 全局编码递增

# 输出编码后的值和位置信息
with open("./encoded_values.txt", "w") as f:
    for block_row_id, pe_id, row, col, access in encoded_values:
        f.write(f"block_row_id: {block_row_id}, PE_id: \
                {pe_id}, Row: {row}, Col: {col}, Code: {access}\n")

block_row_pe_dict = {}
for block_row_id, pe_id, row, col, access in encoded_values:
    if (block_row_id, pe_id) not in block_row_pe_dict:
        block_row_pe_dict[(block_row_id, pe_id)] = []
    block_row_pe_dict[(block_row_id, pe_id)].append(str(access))  # 只保存 code

# 写入文件，每个 block_row_id 和 pe_id 的编码信息占一行
PE_ID = 0
with open("./A_mem_access.txt", "w") as f:
    for (block_row_id, pe_id), access in block_row_pe_dict.items():
        f.write(f"[block_row_id: {block_row_id}, PE_id: {PE_ID}], " +  ", ".join(access) + "\n")
        PE_ID += 1
    

np.set_printoptions(threshold=1000)